In [1]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from pydantic_ai import Agent, RunContext
from pydantic_ai.models import KnownModelName
from pastel.classifiers import classify_insurance_image, classify_images

import logfire
import os
from pastel.helpers import load_images_from_directory
from pastel.models import InsightModel, InsightValidation, InsightPlots
from pastel.prompts import EVALUATION_PROMPT, CLAIMS_VALIDATION_PROMPT

logfire.configure(token=os.getenv("PYDANTIC_LOGFIRE_TOKEN"))
logfire.instrument_openai()
logfire.instrument_anthropic()

OPENAI_MODEL: KnownModelName = "openai:gpt-4o-mini"
ANTHROPIC_MODEL: KnownModelName = "anthropic:claude-3-5-sonnet-latest"

image_dir = "../data/0wvTBiDyAsazB20qDhyomFS505t1ko+Dne-eZsDtqddeA4-SnAMhg8Bctev9RGU1"

In [3]:
validator_agent = Agent(
    model=OPENAI_MODEL,
    deps_type=InsightModel,
    result_type=InsightValidation,
    system_prompt=EVALUATION_PROMPT,
)


@validator_agent.system_prompt
def build_user_prompt(ctx: RunContext[InsightModel]) -> str:
    return CLAIMS_VALIDATION_PROMPT.format(
        program_name=ctx.deps.name,
        line_of_business="EULR",
        insight_text=ctx.deps.insight,
        lrs_summary="",
        images="",
    )

In [4]:
images = InsightPlots(plots=load_images_from_directory(image_dir))

In [ ]:
await classify_images(images)